# Modelling and Evaluating

## Objectives

* Answer business requirement 2:
* The client is interested to tell wheterh a given leaf is covered with powdery mildew

## Inputs

* image shape embeddings (.pkl file)
* Images from the test, train, validation folders and their subfolders.

* inputs/cherryleaves_dataset/cherry-leaves/train
* inputs/cherryleaves_dataset/cherry-leaves/validation
* inputs/cherryleaves_dataset/cherry-leaves/test

## Outputs

* Visualizations:
    * Distribution plots for images in train, validation, and test sets.
    * Bar chart for label distribution.
    * Pie chart for set distribution (percentages).
* Image Augmentation:
    * Apply augmentation techniques.
    * Plot augmented images for each set.
* Data Preparation:
    * Map class indices for prediction inference in labels.
* Model Creation and Training:
    * Create a machine learning model and display its summary.
    * Train the model and save it.
* Plot learning curves for model performance:
    * Model A: separate plots for accuracy and loss.
    * Model B: comprehensive model history plot.
    * Model C: plot model history with Plotly.
* Model Evaluation:
    * Evaluate the model on saved files:
        * Calculate accuracy.
        * Plot ROC curve.
        * Calculate classification report:
            * Model A: classification report.
            * Model B: classification report with macro avg and weighted avg.
            * Model C: synthetic classification report per label.
    * Plot confusion matrix.
    * Save evaluation results in a .pkl file.
* Prediction:
    * Perform prediction on a random image file.

# Comments | Insights | Conclusions

* The same data was plotted in different versions to accomodate possible client's requests of further data understanding.
* The CNN was built seeking maximise accuracy while minimizing loss and training time.
* The CNN was kept as small as possible withouth compromising accuracy and avoiding overfitting.
* More about hyperparameters optimization and trial and error phase is documented in the readme.md file and in a separate .pdf file.

## Import packages

## Set working directory

## Set input directories

Set train, validation and test paths

## Set output directory

## Set Labels

## Set image shape

# Image Distrubtion

# Image data augmentation

## Import ImageDataGenerator

## Initialize ImageDataGenerator

## Set a Batch size

## Augment training image dataset

## Augment validation image dataset

## Augment test image dataset

## Plot augmented training image

## Plot augmented validation and test images

## Save class_indices

# Model creation

## Import model packages

## Model

## Model Summary

## Early Stopping

## Fit model for training

## Save model

## Model Performance

### Model learining curve

## Model Evaluation

### Import saved model

## Evaluate model on test set

## F1 Score

## Save evaluation PKL

# Predict on new data

Load random image as PIL

### Convert image to array and prepare for prediction

### Predict class probabilities